In [1]:
import redis, time, threading
from datetime import datetime

# Redis connection
redis_host = 'localhost'
redis_port = 6379
redis_client = redis.StrictRedis(host=redis_host, port=redis_port, decode_responses=True)

# Chat Room Names
chat_rooms = ['Chat Room 1', 'Chat Room 2', 'Chat Room 3']

def handle_messages(room):
    pubsub = redis_client.pubsub()
    pubsub.subscribe(room)

    for message in pubsub.listen():
        if message['type'] == 'message':
            print(f"[{room}] {message['data']}")

def send_message(room, user_name):

    # Get the current date and time

    # Format the date and time in a beautiful way
    while True:
        message = input(f"Enter your message for [{room}]: ")
        current_datetime = datetime.now().strftime("%A, %d %B %Y %I:%M %p")

        redis_client.publish(room, f"[{current_datetime}-{user_name}]\t{message}")

def join_chat_room(username, room):
    redis_client.sadd(f"users:{room}", username)
    print(f"{username} has joined {room}")

def leave_chat_room(username, room):
    redis_client.srem(f"users:{room}", username)
    print(f"{username} has left {room}")

def main():
    username = input("Enter your username: ")
    print("Available chat rooms:")
    for idx, room in enumerate(chat_rooms, start=1):
        print(f"{idx}. {room}")

    room_choice = int(input("Enter the number of the chat room you want to join: "))
    if 1 <= room_choice <= len(chat_rooms):
        selected_room = chat_rooms[room_choice - 1]
        join_chat_room(username, selected_room)

        # Start message handling thread
        message_thread = threading.Thread(target=handle_messages, args=(selected_room,), daemon=True)
        message_thread.start()

        # Start message sending loop
        send_thread = threading.Thread(target=send_message, args=(selected_room,), daemon=True)
        send_thread.start()

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            leave_chat_room(username, selected_room)



In [2]:
username = input("Enter your username: ")
print("Available chat rooms:")
for idx, room in enumerate(chat_rooms, start=1):
    print(f"{idx}. {room}")

room_choice = int(input("Enter the number of the chat room you want to join: "))
if 1 <= room_choice <= len(chat_rooms):
    selected_room = chat_rooms[room_choice - 1]
    join_chat_room(username, selected_room)

    # Start message handling thread
    message_thread = threading.Thread(target=handle_messages, args=(selected_room,), daemon=True)
    message_thread.start()

    # Start message sending loop
    send_thread = threading.Thread(target=send_message, args=(selected_room,), daemon=True)
    send_thread.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        leave_chat_room(username, selected_room)

Available chat rooms:
1. Chat Room 1
2. Chat Room 2
3. Chat Room 3
duongnka has joined Chat Room 2
[Chat Room 2] hello every body
[Chat Room 2] I am new to this room, it's nice to meet you all
[Chat Room 2] 
[Chat Room 2] 
[Chat Room 2] 
[Chat Room 2] 
[Chat Room 2] 
[Chat Room 2] 


In [3]:
from redis.cluster import RedisCluster as Redis
rc = Redis(host="localhost", port=7001)
rc.get_nodes()

[[host=127.0.0.1,port=7006,name=127.0.0.1:7006,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7006,db=0>>>],
 [host=127.0.0.1,port=7005,name=127.0.0.1:7005,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7005,db=0>>>],
 [host=127.0.0.1,port=7003,name=127.0.0.1:7003,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7003,db=0>>>],
 [host=127.0.0.1,port=7001,name=127.0.0.1:7001,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7001,db=0>>>],
 [host=127.0.0.1,port=7002,name=127.0.0.1:7002,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7002,db=0>>>],
 [host=127.0.0.1,port=7004,name=127.0.0.1:7004,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7004,db=0>>>]]

In [11]:
import sys 
sys.path.append('..')
from utils.redis_utils import *
rc = RedisUtils(RedisType.CLUSTER)
rc.redis_client.get_nodes()

[[host=127.0.0.1,port=7006,name=127.0.0.1:7006,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7006,db=0>>>],
 [host=127.0.0.1,port=7005,name=127.0.0.1:7005,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7005,db=0>>>],
 [host=127.0.0.1,port=7003,name=127.0.0.1:7003,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7003,db=0>>>],
 [host=127.0.0.1,port=7001,name=127.0.0.1:7001,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7001,db=0>>>],
 [host=127.0.0.1,port=7002,name=127.0.0.1:7002,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7002,db=0>>>],
 [host=127.0.0.1,port=7004,name=127.0.0.1:7004,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=127.0.0.1,port=7004,db=0>>>]]

In [13]:
import sys
print(sys.path)

['/Users/duongnka/Learning/Redis/py-redis/rt-chat', '/Users/duongnka/Learning/Redis/py-redis/rt-chat', '/usr/bin/python3', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/duongnka/Library/Python/3.11/lib/python/site-packages', '/opt/homebrew/lib/python3.11/site-packages', '..']
